In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
import pandas as pd
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
user_clr_df = pd.read_csv('/content/drive/MyDrive/5팀_공유폴더/작업결과물/dataset/train/train(CAGR적용_레이블).csv')
test = pd.read_csv('/content/drive/MyDrive/5팀_공유폴더/작업결과물/dataset/test/test(CAGR적용_레이블).csv')
user_clr_df= user_clr_df.set_index('고객번호')
test = test.set_index('고객번호')

# 전체 데이터 평가

In [ ]:
import lightgbm
from lightgbm import LGBMClassifier
X_train=user_clr_df.drop(columns='분기매출증가')
X_train = pd.get_dummies(X_train)

y_train = user_clr_df.분기매출증가

X_test = test.drop(columns='분기매출증가')
X_test = pd.get_dummies(X_test)
y_test = test.분기매출증가



lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
lgbm_wrapper.fit(X_train , y_train)

predict1 = lgbm_wrapper.predict(X_test)
print(accuracy_score(y_test, predict1))

0.6767332610603479


## svm

In [ ]:
X_train=user_clr_df[[ 'fluctuation', 'Monetary',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 

from sklearn.linear_model import SGDClassifier

svm_model = SGDClassifier(random_state=12, loss='log')

svm_model.fit(X_train, y_train)
print(svm_model.score(X_train, y_train))
svm_model.score(X_test, y_test) 
preds = svm_model.predict(X_test)
pred_proba = svm_model.predict_proba(X_test)[:, 1]
get_clf_eval(y_test, preds, pred_proba)

0.7303840594672724
오차 행렬
[[6223 5056]
 [2175 5917]]
정확도: 0.6267, 정밀도: 0.5392, 재현율: 0.7312,    F1: 0.6207, AUC:0.6415


## xgb

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier 

X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 
params = {
    'n_estimators':[100,200],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}

xgb_wrapper = XGBClassifier(random_state=0, n_jobs=-1 )  
grid_cv = GridSearchCV(xgb_wrapper, param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)


print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 200}
최고 예측 정확도: 0.7869


In [ ]:
X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 

# 4 30 30 1000 72
# 6 8 8 200

xgb_wrapper = XGBClassifier(max_depth= 6, min_samples_leaf= 8, min_samples_split= 8, n_estimators= 200)  # merror, mlogloss, auc error
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]
get_clf_eval(y_test , w_preds, w_pred_proba)
print(accuracy_score(y_test , w_preds))



오차 행렬
[[7844 3435]
 [1917 6175]]
정확도: 0.7237, 정밀도: 0.6426, 재현율: 0.7631,    F1: 0.6977, AUC:0.8137
0.7237107015641939


### val

In [ ]:
X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 

# 4 30 30 1000 72
# 6 8 8 200
from sklearn.model_selection import train_test_split

xgb_wrapper = XGBClassifier(max_depth= 6, min_samples_leaf= 8, min_samples_split= 8, n_estimators= 200)  # merror, mlogloss, auc error
xgb_wrapper.fit(X_train, y_train)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,
                                                 test_size=0.3, random_state= 7)
y_pred_train = xgb_wrapper.predict(X_train)
y_pred_val = xgb_wrapper.predict(X_val)
val_score = accuracy_score(y_val, y_pred_val)
print(val_score)


y_pred_test = xgb_wrapper.predict(X_test)
test_score = accuracy_score(y_test, y_pred_test)
test_score       



0.7271163110805231


0.7222509034589571

## lgb

In [ ]:
import lightgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 


params = {
    'n_estimators':[100,200,300,400],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
# 앞서 XGBoost와 동일하게 n_estimators는 400 설정.
lgbm_wrapper = LGBMClassifier(random_state=0, n_jobs=-1)

grid_cv = GridSearchCV(lgbm_wrapper, param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)


print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 300}
최고 예측 정확도: 0.7896


In [ ]:
import lightgbm
from lightgbm import LGBMClassifier
X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 



lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
# lgbm_wrapper.fit(X_train , y_train)

# predict1 = lgbm_wrapper.predict(X_test)
# print(accuracy_score(y_test, predict1))

lgbm_wrapper.fit(X_train, y_train)
w_preds = lgbm_wrapper.predict(X_test)
w_pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]
get_clf_eval(y_test , w_preds, w_pred_proba)
print(accuracy_score(y_test , w_preds))

오차 행렬
[[7879 3400]
 [1883 6209]]
정확도: 0.7273, 정밀도: 0.6462, 재현율: 0.7673,    F1: 0.7015, AUC:0.8159
0.7272727272727273


### val 

In [ ]:
import lightgbm
from lightgbm import LGBMClassifier
X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

from sklearn.model_selection import train_test_split

lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
lgbm_wrapper.fit(X_train , y_train)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,
                                                 test_size=0.3, random_state= 7)
y_pred_train = lgbm_wrapper.predict(X_train)
y_pred_val = lgbm_wrapper.predict(X_val)
val_score = accuracy_score(y_val, y_pred_val)
print(val_score)


y_pred_test = lgbm_wrapper.predict(X_test)
test_score = accuracy_score(y_test, y_pred_test)
test_score         


0.7327942188575362


0.7249059665167048

### scale

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scale= scaler.fit_transform(X_train)
X_test_scale= scaler.transform(X_test)

In [ ]:
lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
lgbm_wrapper.fit(X_scale , y_train)

predict1 = lgbm_wrapper.predict(X_test_scale)
print(accuracy_score(y_test, predict1))

0.7253626555159775


### Pca

In [ ]:
from sklearn.decomposition import PCA

pca_dimension = 6

pca = PCA(n_components=pca_dimension)   # 몇차원으로 압축할 것인가
pca_transformed = pca.fit_transform(X_scale)
pca_transformed = pd.DataFrame(pca_transformed)

In [ ]:
pca_dimension = 6

pca = PCA(n_components=pca_dimension)   # 몇차원으로 압축할 것인가
tpca_transformed = pca.fit_transform(X_test_scale)
tpca_transformed = pd.DataFrame(tpca_transformed)

In [ ]:
lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
lgbm_wrapper.fit(pca_transformed , y_train)
predict1 = lgbm_wrapper.predict(tpca_transformed)
print(accuracy_score(y_test, predict1))

0.654999741882195


### Lda

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1)
lda_transformed = lda.fit_transform(X_scale, y_train)
lda_transformed = pd.DataFrame(lda_transformed)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=1)
lda_transformed_t = lda.fit_transform(X_test_scale, y_test)
lda_transformed_t = pd.DataFrame(lda_transformed_t)

In [ ]:
lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
lgbm_wrapper.fit(lda_transformed , y_train)
predict1 = lgbm_wrapper.predict(lda_transformed_t)
print(accuracy_score(y_test, predict1))

0.724175313613133


### svd

In [ ]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=6)
tsvd_transformed = tsvd.fit_transform(X_train)
tsvd_transformed = pd.DataFrame(tsvd_transformed)

In [ ]:
tsvd = TruncatedSVD(n_components=6)
tsvd_transformed_t = tsvd.fit_transform(X_test)
tsvd_transformed_t = pd.DataFrame(tsvd_transformed_t)

In [ ]:
lgbm_wrapper = LGBMClassifier(max_depth= 9, min_samples_leaf=2 , min_samples_split=4, n_estimators = 300)
lgbm_wrapper.fit(tsvd_transformed , y_train)
predict1 = lgbm_wrapper.predict(tsvd_transformed_t)
print(accuracy_score(y_test, predict1))

0.718083733415931


## 디시젼

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 

from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
pipe_tree = make_pipeline(DecisionTreeClassifier(random_state=0))
from sklearn.model_selection import GridSearchCV
 
param_range1 = [1,2,3,4,5,6,7,8,9,10]
param_range2 = [10, 20, 30, 40, 50]
param_range3 = ['gini', 'entropy']
 
param_grid = [{'decisiontreeclassifier__max_depth': param_range1,
              'decisiontreeclassifier__min_samples_leaf': param_range2,
              'decisiontreeclassifier__criterion': param_range3}]
 
gs = GridSearchCV(estimator = pipe_tree,
                 param_grid = param_grid, # 찾고자하는 파라미터. dictionary 형식
                 scoring = 'accuracy', #  Classification일때  'accuracy','f1', Regression 일때 'neg_mean_squared_error','r2' 등
                 cv=10,
                 n_jobs= -1) # 병렬 처리갯수? -1은 전부를 의미
 
gs = gs.fit(X_train, y_train)
 
print(gs.best_score_)
print(gs.best_params_)


In [ ]:
from sklearn.tree import DecisionTreeClassifier # 59

X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 

from sklearn.tree import DecisionTreeClassifier # 59

df_clf = DecisionTreeClassifier(max_depth= 6, min_samples_leaf=8 ,criterion = 'entropy')  

df_clf.fit(X_train, y_train)

w_preds = df_clf.predict(X_test)
w_pred_proba = df_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test , w_preds, w_pred_proba)
print(accuracy_score(y_test , w_preds))

오차 행렬
[[7436 3843]
 [1970 6122]]
정확도: 0.6999, 정밀도: 0.6144, 재현율: 0.7565,    F1: 0.6781, AUC:0.7709
0.6999122399463115


## 로지스틱

In [ ]:
X_train=user_clr_df[[ 'Recency', 'Frequency', 'Monetary', 
      'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
       'D_Monetary_실제지출',  'A_Monetary_비율',
       'B_Monetary_비율', 'C_Monetary_비율', 'D_Monetary_비율','명품구매자', '구매횟수','A_fluctuation', 'B_fluctuation', 'C_fluctuation',
       'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '준내구_fluctuation', '내구_fluctuation',
       '비내구_fluctuation', '선매_fluctuation', '전문_fluctuation', '신규고객', 'A_recency']]
y_train = user_clr_df.분기매출증가
X_test = test[['Recency', 'Frequency', 'Monetary',  
      'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
       'D_Monetary_실제지출',  'A_Monetary_비율',
       'B_Monetary_비율', 'C_Monetary_비율', 'D_Monetary_비율','명품구매자', '구매횟수','A_fluctuation', 'B_fluctuation', 'C_fluctuation',
       'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '준내구_fluctuation', '내구_fluctuation',
       '비내구_fluctuation', '선매_fluctuation', '전문_fluctuation', '신규고객','A_recency']]

y_test = test.분기매출증가 

from sklearn.linear_model import LogisticRegression 
model = LogisticRegression()
model.fit(X_train, y_train)
w_preds = model.predict(X_test)
w_pred_proba = model.predict_proba(X_test)[:, 1]
get_clf_eval(y_test , w_preds, w_pred_proba)
print(accuracy_score(y_test , w_preds))

오차 행렬
[[4024 7255]
 [1640 6452]]
정확도: 0.5408, 정밀도: 0.4707, 재현율: 0.7973,    F1: 0.5920, AUC:0.6601
0.5408084249651541


## 랜덤

In [ ]:
X_train=user_clr_df[[ 'fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]
y_train = user_clr_df.분기매출증가
X_test = test[['fluctuation', 'Monetary', 'cagr',
'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
'D_Monetary_실제지출',
'A_fluctuation', 'B_fluctuation', 'C_fluctuation',
  'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '선매_fluctuation', '전문_fluctuation']]

y_test = test.분기매출증가 

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100,200, 400],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
# # RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 


X_train=user_clr_df[[ 'Recency', 'Frequency', 'Monetary', 
      'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
       'D_Monetary_실제지출',  'A_Monetary_비율',
       'B_Monetary_비율', 'C_Monetary_비율', 'D_Monetary_비율','명품구매자', '구매횟수','A_fluctuation', 'B_fluctuation', 'C_fluctuation',
       'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '준내구_fluctuation', '내구_fluctuation',
       '비내구_fluctuation', '선매_fluctuation', '전문_fluctuation', '신규고객', 'A_recency']]
y_train = user_clr_df.분기매출증가
X_test = test[['Recency', 'Frequency', 'Monetary',  
      'A_Monetary_실제지출', 'B_Monetary_실제지출', 'C_Monetary_실제지출',
       'D_Monetary_실제지출',  'A_Monetary_비율',
       'B_Monetary_비율', 'C_Monetary_비율', 'D_Monetary_비율','명품구매자', '구매횟수','A_fluctuation', 'B_fluctuation', 'C_fluctuation',
       'D_fluctuation', '고급_fluctuation', '일반_fluctuation',
       '편의_fluctuation', '준내구_fluctuation', '내구_fluctuation',
       '비내구_fluctuation', '선매_fluctuation', '전문_fluctuation', '신규고객','A_recency']]

y_test = test.분기매출증가 

from sklearn.ensemble import RandomForestClassifier # 53 # 58

clf = RandomForestClassifier(max_depth= 10, min_samples_leaf= 18, min_samples_split= 8, n_estimators=400)
clf.fit(X_train, y_train)
w_preds = clf.predict(X_test)
w_pred_proba = clf.predict_proba(X_test)[:, 1]
get_clf_eval(y_test , w_preds, w_pred_proba)
print(accuracy_score(y_test , w_preds))



오차 행렬
[[6943 4336]
 [1917 6175]]
정확도: 0.6772, 정밀도: 0.5875, 재현율: 0.7631,    F1: 0.6639, AUC:0.7706
0.677197873109287


# 파이캐럿

In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,677 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from datetime import datetime

from keras import layers
from keras import Sequential
from keras import optimizers
from keras.layers import Input, Dense, Flatten, Embedding

from keras.utils import plot_model
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import plotly.express as px

# 한글 폰트 출력
plt.rc('font', family='NanumBarunGothic') 

RuntimeError: ignored

ImportError: ignored

In [ ]:
from pycaret.classification import *
exp_101 = setup(data=test, target='분기매출증가',
                session_id=123, fold_shuffle=True)

,Description,Value
0,session_id,123
1,Target,분기매출증가
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(19371, 130)"
5,Missing Values,False
6,Numeric Features,112
7,Categorical Features,17
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=['Unnamed: 0'],
                                      ml_usecase='classification',
                                      numerical_features=[], target='분기매출증가',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                nu...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('clust

In [ ]:
best_model = compare_models() 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7774,0.8552,0.6919,0.7517,0.7204,0.5361,0.5375,1.167
gbc,Gradient Boosting Classifier,0.7698,0.8463,0.6690,0.7493,0.7066,0.5183,0.5207,13.794
ridge,Ridge Classifier,0.7571,0.0000,0.6179,0.7524,0.6782,0.4861,0.4925,0.056
lda,Linear Discriminant Analysis,0.7569,0.8325,0.6197,0.7509,0.6787,0.4861,0.4922,0.473
ada,Ada Boost Classifier,0.7546,0.8274,0.6430,0.7329,0.6846,0.4853,0.4884,3.210
rf,Random Forest Classifier,0.7542,0.8252,0.6346,0.7364,0.6816,0.4832,0.4869,4.467
et,Extra Trees Classifier,0.7487,0.8147,0.6167,0.7349,0.6704,0.4698,0.4747,3.151
dt,Decision Tree Classifier,0.6782,0.6697,0.6197,0.6104,0.6150,0.3386,0.3387,0.832
lr,Logistic Regression,0.6526,0.6618,0.3392,0.6577,0.4472,0.2299,0.2570,0.534
svm,SVM - Linear Kernel,0.6214,0.0000,0.3737,0.6600,0.3919,0.1775,0.2196,0.200


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:compare_models() succesfully completed......................................


In [ ]:
evaluate_model(best_model)# 피처중요도 보는 것 

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…